In [ ]:
import torch

torch.manual_seed(42)
torch.cuda.is_available()

In [ ]:
import pandas as pd
df = pd.read_csv('sample_data/fashion-mnist_train.csv')
df.shape

In [ ]:
from matplotlib import pyplot as plt


fig, axes = plt.subplots(4, 4, figsize=(10, 10))
fig.suptitle("First 16 images", fontsize=16)

for i, ax in enumerate(axes.flat):
    img = df.iloc[i, 1:].values.reshape(28, 28)
    ax.imshow(img)
    ax.axis("off")
    ax.set_title(df.iloc[i, 0])

plt.show()

In [ ]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

# If removed .values from this line it will return a Pandas DataFrame instead of a NumPy array.

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

In [ ]:
train_dataset = CustomDataset(X_train, y_train)
len(train_dataset)

In [ ]:
test_dataset = CustomDataset(X_test, y_test)

from torch.utils.data import DataLoader


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
import torch.nn as nn


class NN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
epochs = 100
learning_rate = 0.1

model = NN(X_train.shape[1])
model = model.to("cuda")
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [ ]:
for epoch in range(epochs):
    total_loss = 0
    for batch_features, batch_labels in train_loader:
        batch_features = batch_features.to("cuda")
        batch_labels = batch_labels.to("cuda")
        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch: {epoch+1}, Loss: {avg_loss:.4f}")

In [ ]:
model.eval()
total = correct = 0

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        batch_features, batch_labels = batch_features.to("cuda"), batch_labels.to("cuda")
        outputs = model(batch_features)
        _, predicted = torch.max(outputs, 1)
        # print(predicted, batch_labels)
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()
    print(correct/total)

In [ ]:
model.eval()
total = correct = 0

with torch.no_grad():
    for batch_features, batch_labels in train_loader:
        batch_features, batch_labels = batch_features.to("cuda"), batch_labels.to("cuda")
        outputs = model(batch_features)
        _, predicted = torch.max(outputs, 1)
        # print(predicted, batch_labels)
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()
    print(correct/total)